In [1]:
import torch
import torch.nn.functional as F

In [263]:
q_test = torch.randn((1, 1, 8, 2))
k_test = torch.randn((1, 1, 8, 2))
v_test = torch.randn((1, 8, 1, 2)) # tem o shape diferente sei la porque

In [15]:
# primeira coisa e dar um jeito de criar os chunks
window = 128

In [16]:
q = q_test.view(q_test.size(0), q_test.size(1), q_test.size(2) // (window * 2), window * 2, q_test.size(3))
q.shape

torch.Size([2, 8, 2, 256, 32])

In [17]:
chunk_size = list(q.size())
chunk_size[2] = chunk_size[2] * 2 - 1
chunk_size

[2, 8, 3, 256, 32]

In [18]:
chunk_stride = list(q.stride())
chunk_stride[2] = chunk_stride[2] // 2
chunk_stride

[131072, 16384, 4096, 32, 1]

In [19]:
q_strided = q.as_strided(size=chunk_size, stride=chunk_stride)
q_strided.shape

torch.Size([2, 8, 3, 256, 32])

In [20]:
q

tensor([[[[[ 1.4142e-01, -3.4586e-02,  4.8907e-01,  ...,  1.0610e+00,
            -1.6854e+00, -2.6077e-01],
           [ 5.7260e-01,  1.3921e+00,  8.4185e-01,  ..., -7.1958e-02,
             4.1668e-01,  7.4212e-01],
           [ 3.7492e-01, -1.1776e+00, -1.2224e+00,  ...,  8.7843e-01,
            -1.1105e-01, -1.8354e+00],
           ...,
           [ 2.6437e-02,  5.2077e-01,  5.9023e-01,  ..., -9.0006e-01,
             1.4149e-01, -1.2045e+00],
           [ 1.7731e-01, -7.6056e-03, -7.7332e-01,  ..., -3.2441e-01,
            -9.2750e-01, -7.3691e-01],
           [-8.4653e-01,  8.2502e-01,  1.9588e+00,  ...,  4.4917e-01,
            -6.1399e-01, -1.5528e-01]],

          [[ 2.5383e-01, -1.1661e+00,  1.3393e+00,  ..., -6.4339e-01,
            -4.0338e-01,  1.2171e-01],
           [-1.8586e+00,  2.1447e-01,  4.9410e-01,  ..., -1.4613e-01,
            -4.6260e-01,  3.3066e-01],
           [-4.5290e-01, -1.0059e+00,  2.6501e-01,  ..., -1.1389e+00,
            -9.9486e-01,  1.6171e+00],
 

In [21]:
q_strided

tensor([[[[[ 1.4142e-01, -3.4586e-02,  4.8907e-01,  ...,  1.0610e+00,
            -1.6854e+00, -2.6077e-01],
           [ 5.7260e-01,  1.3921e+00,  8.4185e-01,  ..., -7.1958e-02,
             4.1668e-01,  7.4212e-01],
           [ 3.7492e-01, -1.1776e+00, -1.2224e+00,  ...,  8.7843e-01,
            -1.1105e-01, -1.8354e+00],
           ...,
           [ 2.6437e-02,  5.2077e-01,  5.9023e-01,  ..., -9.0006e-01,
             1.4149e-01, -1.2045e+00],
           [ 1.7731e-01, -7.6056e-03, -7.7332e-01,  ..., -3.2441e-01,
            -9.2750e-01, -7.3691e-01],
           [-8.4653e-01,  8.2502e-01,  1.9588e+00,  ...,  4.4917e-01,
            -6.1399e-01, -1.5528e-01]],

          [[ 1.9752e+00,  1.5118e+00,  6.8340e-03,  ..., -6.7781e-01,
            -2.5607e-01,  5.4671e-02],
           [ 4.4032e-01, -7.2030e-01,  4.3117e-01,  ...,  5.4652e-01,
            -3.8645e-03,  1.1430e+00],
           [ 2.6060e+00,  5.3271e-01, -7.4680e-01,  ...,  8.0288e-01,
             2.1611e+00, -6.5659e-01],
 

In [166]:

def _chunk(x, w):
    '''convert into overlapping chunkings. Chunk size = 2w, overlap size = w'''

    # non-overlapping chunks of size = 2w
    x = x.view(x.size(0), x.size(1), x.size(2) // (w * 2), w * 2, x.size(3))

    # use `as_strided` to make the chunks overlap with an overlap size = w
    chunk_size = list(x.size())
    chunk_size[2] = chunk_size[2] * 2 - 1

    chunk_stride = list(x.stride())
    chunk_stride[2] = chunk_stride[2] // 2
    return x.as_strided(size=chunk_size, stride=chunk_stride)

In [264]:
w = 2
bsz, num_heads, seqlen, head_dim = q_test.size()
chunks_count = seqlen // w - 1
print(q_test.size())

torch.Size([1, 1, 8, 2])


In [265]:
assert seqlen % (w * 2) == 0

In [305]:
q_chunk = _chunk(q_test, w)
k_chunk = _chunk(k_test, w)

In [306]:
q_chunk.shape

torch.Size([1, 1, 3, 4, 2])

In [307]:
q_test

tensor([[[[ 1.1450,  1.0662],
          [-0.6897, -0.9464],
          [-0.2917, -0.5592],
          [-0.5090, -1.7635],
          [-0.2397, -1.8315],
          [ 0.5622,  2.8601],
          [-0.2709, -0.2056],
          [-1.6546,  0.2921]]]])

In [308]:
q_chunk

tensor([[[[[ 1.1450,  1.0662],
           [-0.6897, -0.9464],
           [-0.2917, -0.5592],
           [-0.5090, -1.7635]],

          [[-0.2917, -0.5592],
           [-0.5090, -1.7635],
           [-0.2397, -1.8315],
           [ 0.5622,  2.8601]],

          [[-0.2397, -1.8315],
           [ 0.5622,  2.8601],
           [-0.2709, -0.2056],
           [-1.6546,  0.2921]]]]])

In [309]:
k_chunk.shape

torch.Size([1, 1, 3, 4, 2])

In [324]:
chunk_att = torch.einsum("bhcxd,bhcyd->bhcxy", (q_chunk, k_chunk))
chunk_att.shape

torch.Size([1, 1, 3, 4, 4])

In [325]:
def _skew(x, direction, padding_value):
    '''Convert diagonals into columns (or columns into diagonals depending on `direction`'''
    x_padded = F.pad(x, direction, value=padding_value)
    x_padded = x_padded.view(*x_padded.size()[:-2], x_padded.size(-1), x_padded.size(-2))
    return x_padded

In [326]:
diagonal_chunk_att = _skew(chunk_att, direction=(0, 0, 0, 1), padding_value=0)
diagonal_chunk_att.shape

torch.Size([1, 1, 3, 4, 5])

In [327]:
diagonal_attn = diagonal_chunk_att.new_empty((bsz, num_heads, chunks_count + 1, w, w * 2 + 1))

In [328]:
diagonal_attn.shape

torch.Size([1, 1, 4, 2, 5])

In [329]:
# copy parts from diagonal_chunk_attn into the compined matrix of attentions
# - copying the main diagonal and the upper triangle
diagonal_attn[:, :, :-1, :, w:] = diagonal_chunk_att[:, :, :, :w, :w + 1]
diagonal_attn[:, :, -1, :, w:] = diagonal_chunk_att[:, :, -1, w:, :w + 1]
# - copying the lower triangle
diagonal_attn[:, :, 1:, :, :w] = diagonal_chunk_att[:, :, :, - (w + 1):-1, w + 1:]
diagonal_attn[:, :, 0, 1:w, 1:w] = diagonal_chunk_att[:, :, 0, :w - 1, 1 - w:]
diagonal_attn

tensor([[[[[ 3.6818e-35,  0.0000e+00,  2.2290e-01,  9.2639e-01,  2.3645e+00],
           [ 0.0000e+00, -9.2330e-02, -4.9862e-01, -2.0144e+00, -3.9699e-01]],

          [[-1.7157e-02, -1.7990e-01, -1.1602e+00, -2.6297e-01,  1.0173e+00],
           [-1.5994e-01, -3.5528e+00, -9.3031e-01,  2.6888e+00, -1.9794e+00]],

          [[-3.6152e+00, -1.0372e+00,  2.4275e+00, -2.2839e+00, -2.2283e+00],
           [ 1.5735e+00, -4.0281e+00,  3.4182e+00,  3.5350e+00,  4.9458e+00]],

          [[ 5.8090e-01, -6.3687e-02, -3.2206e-01, -7.4770e-01,  1.7524e+00],
           [ 1.7009e+00, -1.4371e-01, -2.4076e+00,  0.0000e+00,  0.0000e+00]]]]])

In [330]:
diagonal_attn = diagonal_attn.reshape(bsz, num_heads, seqlen, 2*w + 1)
diagonal_attn.shape

torch.Size([1, 1, 8, 5])

In [331]:
diagonal_attn

tensor([[[[ 3.6818e-35,  0.0000e+00,  2.2290e-01,  9.2639e-01,  2.3645e+00],
          [ 0.0000e+00, -9.2330e-02, -4.9862e-01, -2.0144e+00, -3.9699e-01],
          [-1.7157e-02, -1.7990e-01, -1.1602e+00, -2.6297e-01,  1.0173e+00],
          [-1.5994e-01, -3.5528e+00, -9.3031e-01,  2.6888e+00, -1.9794e+00],
          [-3.6152e+00, -1.0372e+00,  2.4275e+00, -2.2839e+00, -2.2283e+00],
          [ 1.5735e+00, -4.0281e+00,  3.4182e+00,  3.5350e+00,  4.9458e+00],
          [ 5.8090e-01, -6.3687e-02, -3.2206e-01, -7.4770e-01,  1.7524e+00],
          [ 1.7009e+00, -1.4371e-01, -2.4076e+00,  0.0000e+00,  0.0000e+00]]]])

A matriz gerada é exatamente o que precisavamos. Cada linha já é a atenção do token para os 2 * w vizinhos, só que num formato de coluna.

In [332]:
# agora vamos ver o value
# os dois primeiros valores e dois ultimos precisam ser -inf
# chunk_prob = diagonal_attn
# chunk_prob[:, :, -1, -2:] = float("-inf")
# chunk_prob[:, :, 0, :2] = float("-inf")
# chunk_prob = torch.softmax(chunk_prob, dim=-1)
# chunk_prob

In [334]:
# mascara atencoes invalidas
chunk_prob = diagonal_attn
for i in range(w):
    # tira da esquerda
    chunk_prob[:, :, i, :w - i] = float("-inf")
    # tira da direita
    chunk_prob[:, :, -(i + 1), -(w - i):] = float("-inf")
print(chunk_prob)

tensor([[[[   -inf,    -inf,  0.2229,  0.9264,  2.3645],
          [   -inf, -0.0923, -0.4986, -2.0144, -0.3970],
          [-0.0172, -0.1799, -1.1602, -0.2630,  1.0173],
          [-0.1599, -3.5528, -0.9303,  2.6888, -1.9794],
          [-3.6152, -1.0372,  2.4275, -2.2839, -2.2283],
          [ 1.5735, -4.0281,  3.4182,  3.5350,  4.9458],
          [ 0.5809, -0.0637, -0.3221, -0.7477,    -inf],
          [ 1.7009, -0.1437, -2.4076,    -inf,    -inf]]]])


In [280]:
chunk_prob = chunk_prob.reshape(bsz * num_heads, seqlen // w, w, 2 * w + 1)
chunk_prob

tensor([[[[0.0000e+00, 0.0000e+00, 8.6701e-02, 1.7520e-01, 7.3810e-01],
          [3.0076e-01, 2.7423e-01, 1.8267e-01, 4.0120e-02, 2.0221e-01]],

         [[1.7348e-01, 1.4742e-01, 5.5312e-02, 1.3567e-01, 4.8811e-01],
          [5.2841e-02, 1.7761e-03, 2.4457e-02, 9.1236e-01, 8.5661e-03]],

         [[2.2574e-03, 2.9734e-02, 9.5043e-01, 8.5467e-03, 9.0357e-03],
          [2.2944e-02, 8.4703e-05, 1.4515e-01, 1.6313e-01, 6.6869e-01]],

         [[1.8443e-01, 9.6804e-02, 7.4762e-02, 4.8846e-02, 5.9515e-01],
          [8.5142e-01, 1.3459e-01, 1.3990e-02, 0.0000e+00, 0.0000e+00]]]])

In [281]:
# comeca colocando um padding no value. W no comeco e W no final
bsz, seqlen, num_heads, head_dim = v_test.size()
v = v_test.transpose(2, 1).reshape((bsz * num_heads, seqlen, head_dim))
padded_v = F.pad(v, (0, 0, w, w), value=-1)
padded_v

tensor([[[-1.0000, -1.0000],
         [-1.0000, -1.0000],
         [-0.9510,  0.4976],
         [ 0.9218,  0.3977],
         [ 0.6368, -1.6312],
         [ 0.1111, -0.1860],
         [ 0.3590, -1.1337],
         [ 0.3291,  0.4823],
         [ 1.4909,  1.3430],
         [-0.1302, -0.4666],
         [-1.0000, -1.0000],
         [-1.0000, -1.0000]]])

In [252]:
padded_v.shape

torch.Size([1, 12, 2])

In [253]:
# chunk padded_v into chunks of size 3w and an overlap of size w
chunks_count = seqlen // w - 1
chunk_v_size = (bsz * num_heads, chunks_count + 1, 3* w, head_dim)
chunk_v_stride = padded_v.stride()
chunk_v_stride = chunk_v_stride[0], w * chunk_v_stride[1], chunk_v_stride[1], chunk_v_stride[2]
chunk_v = padded_v.as_strided(size=chunk_v_size, stride=chunk_v_stride)
chunk_v

tensor([[[[-1.0000, -1.0000],
          [-1.0000, -1.0000],
          [-0.4686,  1.4253],
          [-0.5567,  0.2106],
          [-0.2623, -1.2736],
          [ 2.1774,  0.4892]],

         [[-0.4686,  1.4253],
          [-0.5567,  0.2106],
          [-0.2623, -1.2736],
          [ 2.1774,  0.4892],
          [ 0.9090,  0.9877],
          [ 0.4194,  0.3343]],

         [[-0.2623, -1.2736],
          [ 2.1774,  0.4892],
          [ 0.9090,  0.9877],
          [ 0.4194,  0.3343],
          [ 0.9080, -0.6235],
          [ 0.0095, -0.4979]],

         [[ 0.9090,  0.9877],
          [ 0.4194,  0.3343],
          [ 0.9080, -0.6235],
          [ 0.0095, -0.4979],
          [-1.0000, -1.0000],
          [-1.0000, -1.0000]]]])

In [254]:
chunk_v.shape

torch.Size([1, 4, 6, 2])

In [255]:
chunk_v_stride

(24, 4, 2, 1)

In [256]:
chunk_v_size

(1, 4, 6, 2)

In [257]:
def _skew2(x, padding_value):
    '''shift every row 1 step to right converting columns into diagonals'''
    # X = B x C x M x L
    B, C, M, L = x.size()
    x = F.pad(x, (0, M + 1), value=padding_value)  # B x C x M x (L+M+1)
    x = x.view(B, C, -1)  # B x C x ML+MM+M
    x = x[:, :, :-M]  # B x C x ML+MM
    x = x.view(B, C, M, M + L)  # B x C, M x L+M
    x = x[:, :, :, :-1]
    return x


In [258]:
skewed_prob = _skew2(chunk_prob, padding_value=0)
print(skewed_prob.shape)
skewed_prob

torch.Size([1, 4, 2, 6])


tensor([[[[0.0000, 0.0000, 0.0468, 0.4135, 0.5397, 0.0000],
          [0.0000, 0.1278, 0.3223, 0.2840, 0.0533, 0.2127]],

         [[0.0642, 0.2070, 0.2734, 0.1330, 0.3224, 0.0000],
          [0.0000, 0.0187, 0.7002, 0.1811, 0.0370, 0.0631]],

         [[0.0519, 0.0753, 0.3070, 0.2120, 0.3538, 0.0000],
          [0.0000, 0.2129, 0.3166, 0.2299, 0.1316, 0.1090]],

         [[0.2546, 0.3329, 0.0718, 0.0763, 0.2644, 0.0000],
          [0.0000, 0.0694, 0.3918, 0.5388, 0.0000, 0.0000]]]])

In [259]:
context = torch.einsum("bcwd,bcdh->bcwh", (skewed_prob, chunk_v))
print(context.shape)
context

torch.Size([1, 4, 2, 2])


tensor([[[[-0.3937, -0.5335],
          [ 0.0122,  0.4276]],

         [[ 0.3655,  0.1704],
          [ 0.2602, -0.7416]],

         [[ 0.8395,  0.1243],
          [ 0.9684,  0.3575]],

         [[ 0.1726,  0.0157],
          [ 0.3900, -0.4894]]]])

In [260]:
context = context.view((bsz, num_heads, seqlen, head_dim)).transpose(1, 2)
context

tensor([[[[-0.3937, -0.5335]],

         [[ 0.0122,  0.4276]],

         [[ 0.3655,  0.1704]],

         [[ 0.2602, -0.7416]],

         [[ 0.8395,  0.1243]],

         [[ 0.9684,  0.3575]],

         [[ 0.1726,  0.0157]],

         [[ 0.3900, -0.4894]]]])

In [261]:
context.shape

torch.Size([1, 8, 1, 2])

In [335]:
diagonal_attn

tensor([[[[   -inf,    -inf,  0.2229,  0.9264,  2.3645],
          [   -inf, -0.0923, -0.4986, -2.0144, -0.3970],
          [-0.0172, -0.1799, -1.1602, -0.2630,  1.0173],
          [-0.1599, -3.5528, -0.9303,  2.6888, -1.9794],
          [-3.6152, -1.0372,  2.4275, -2.2839, -2.2283],
          [ 1.5735, -4.0281,  3.4182,  3.5350,  4.9458],
          [ 0.5809, -0.0637, -0.3221, -0.7477,    -inf],
          [ 1.7009, -0.1437, -2.4076,    -inf,    -inf]]]])

In [283]:
diagonal_attn.shape

torch.Size([1, 1, 8, 5])

In [357]:
# como fica o positional encoding?
context_position = torch.arange(end=w + 1,dtype=torch.long, start=-w)[
    None, :
]
memory_position = torch.arange(seqlen, dtype=torch.long)[
    :, None
]
relative_position = memory_position + context_position
relative_position = relative_position.abs() * -1
relative_position

tensor([[-2, -1,  0, -1, -2],
        [-1,  0, -1, -2, -3],
        [ 0, -1, -2, -3, -4],
        [-1, -2, -3, -4, -5],
        [-2, -3, -4, -5, -6],
        [-3, -4, -5, -6, -7],
        [-4, -5, -6, -7, -8],
        [-5, -6, -7, -8, -9]])

In [358]:
import math
def get_exponential_slopes(n):
    sizes = [64, 128, 255, 512, 1024, 2048, 4096, 8192, 16384, 32768]
    slopes = []
    for i in range(n):
        usavel = i
        while usavel >= len(sizes):
            usavel -= len(sizes)
        slopes.append(math.log(0.1) / sizes[usavel])
    return slopes

In [359]:
pos_encoding = relative_position.unsqueeze(0).expand(num_heads, -1, -1)
print(pos_encoding.shape)
pos_encoding

torch.Size([1, 8, 5])


tensor([[[-2, -1,  0, -1, -2],
         [-1,  0, -1, -2, -3],
         [ 0, -1, -2, -3, -4],
         [-1, -2, -3, -4, -5],
         [-2, -3, -4, -5, -6],
         [-3, -4, -5, -6, -7],
         [-4, -5, -6, -7, -8],
         [-5, -6, -7, -8, -9]]])

In [360]:
slopes = torch.Tensor(get_exponential_slopes(num_heads))
slopes = slopes.unsqueeze(1).unsqueeze(1)
pos_encoding_mask = torch.exp(pos_encoding * torch.abs(slopes))
pos_encoding_mask

tensor([[[0.9306, 0.9647, 1.0000, 0.9647, 0.9306],
         [0.9647, 1.0000, 0.9647, 0.9306, 0.8977],
         [1.0000, 0.9647, 0.9306, 0.8977, 0.8660],
         [0.9647, 0.9306, 0.8977, 0.8660, 0.8354],
         [0.9306, 0.8977, 0.8660, 0.8354, 0.8058],
         [0.8977, 0.8660, 0.8354, 0.8058, 0.7774],
         [0.8660, 0.8354, 0.8058, 0.7774, 0.7499],
         [0.8354, 0.8058, 0.7774, 0.7499, 0.7234]]])

In [363]:
pos_encoding_mask.shape

torch.Size([1, 8, 5])

In [364]:
diagonal_attn

tensor([[[[   -inf,    -inf,  0.2229,  0.9264,  2.3645],
          [   -inf, -0.0923, -0.4986, -2.0144, -0.3970],
          [-0.0172, -0.1799, -1.1602, -0.2630,  1.0173],
          [-0.1599, -3.5528, -0.9303,  2.6888, -1.9794],
          [-3.6152, -1.0372,  2.4275, -2.2839, -2.2283],
          [ 1.5735, -4.0281,  3.4182,  3.5350,  4.9458],
          [ 0.5809, -0.0637, -0.3221, -0.7477,    -inf],
          [ 1.7009, -0.1437, -2.4076,    -inf,    -inf]]]])

In [365]:
diagonal_attn + pos_encoding_mask

tensor([[[[   -inf,    -inf,  1.2229,  1.8911,  3.2951],
          [   -inf,  0.9077,  0.4660, -1.0839,  0.5007],
          [ 0.9828,  0.7848, -0.2296,  0.6347,  1.8833],
          [ 0.8047, -2.6222, -0.0326,  3.5548, -1.1441],
          [-2.6846, -0.1395,  3.2934, -1.4485, -1.4224],
          [ 2.4712, -3.1622,  4.2536,  4.3409,  5.7231],
          [ 1.4469,  0.7717,  0.4838,  0.0297,    -inf],
          [ 2.5363,  0.6621, -1.6303,    -inf,    -inf]]]])

# MEU DEUS FUNCIONOU!

In [151]:
nova = torch.ones((bsz, num_heads, seqlen, seqlen)) * float("-inf")
nova

tensor([[[[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf]]]])

In [152]:
chunk_att.shape

torch.Size([1, 1, 3, 4, 4])

In [153]:
num_chunks = chunk_att.size(2)
for i in range(num_chunks):
    print(i * w)
    print((i + 2) * w)
    nova[:, :, i * w: (i + 2) * w, i*w: (i + 2) * w] = chunk_att[:, :, i, :, :]
    print(nova)

0
4
tensor([[[[ 1.1603,  1.1672, -1.1073,  0.2787,    -inf,    -inf,    -inf,
              -inf],
          [ 0.6695,  1.3846, -1.0546,  0.5635,    -inf,    -inf,    -inf,
              -inf],
          [-0.4201, -1.5400,  1.0541, -0.7338,    -inf,    -inf,    -inf,
              -inf],
          [ 1.2405, -1.0459,  0.1570, -1.0012,    -inf,    -inf,    -inf,
              -inf],
          [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,
              -inf],
          [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,
              -inf],
          [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,
              -inf],
          [   -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,
              -inf]]]])
2
6
tensor([[[[ 1.1603,  1.1672, -1.1073,  0.2787,    -inf,    -inf,    -inf,
              -inf],
          [ 0.6695,  1.3846, -1.0546,  0.5635,    -inf,    -inf,    -inf,
              -inf],
          [-0.4201, -1.5400,  1.0541, -

In [149]:
nova

tensor([[[[ 1.1603,  1.1672, -1.1073,  0.2787,  0.0000,  0.0000,  0.0000,
            0.0000],
          [ 0.6695,  1.3846, -1.0546,  0.5635,  0.0000,  0.0000,  0.0000,
            0.0000],
          [-0.4201, -1.5400,  1.0541, -0.7338, -3.0537, -0.6180,  0.0000,
            0.0000],
          [ 1.2405, -1.0459,  0.1570, -1.0012, -0.9655,  0.0323,  0.0000,
            0.0000],
          [ 0.0000,  0.0000, -1.1294,  0.4519,  3.0804,  0.7087, -0.9454,
           -1.2840],
          [ 0.0000,  0.0000,  0.5368,  2.1687, -0.0994, -0.6690, -0.9121,
            1.6827],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  2.4977,  0.5848, -0.7518,
           -1.0668],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.9735,  0.2564, -0.2515,
           -0.4881]]]])